In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_data(poll_data, low_memory=False):
  poll_data= pd.read_csv(poll_data)
  return  poll_data

In [ ]:
poll_data = '/content/drive/MyDrive/president_polls_historical.csv'
poll_data = load_data(poll_data)
poll_data.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,73830,940,Lake Research,1587,National Women's Law Center,Lake Research Partners,165,Lake Research Partners,1.2,0.1,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,51.0
1,73830,940,Lake Research,1587,National Women's Law Center,Lake Research Partners,165,Lake Research Partners,1.2,0.1,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,48.0
2,74946,1677,The Political Matrix/The Listener Group,NaN,NaN,The Political Matrix/The Listener Group,677,The Political Matrix/The Listener Group,0.7,0.7,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,44.2
3,74946,1677,The Political Matrix/The Listener Group,NaN,NaN,The Political Matrix/The Listener Group,677,The Political Matrix/The Listener Group,0.7,0.7,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,48.0
4,72621,383,PPP,NaN,NaN,Public Policy Polling,263,Public Policy Polling,1.4,0.0,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,49.0


In [ ]:
poll_data.shape

(16715, 48)

In [ ]:
def select_relevant_columns(poll_data):
    relevant_columns = ['pollster', 'start_date', 'end_date', 'party', 'candidate_name', 'pct']
    return poll_data[relevant_columns]

In [ ]:
poll_data = select_relevant_columns(poll_data)

In [ ]:
poll_data.head()

,pollster,start_date,end_date,party,candidate_name,pct
0,Lake Research,10/31/20,11/3/20,DEM,Joe Biden,51.0
1,Lake Research,10/31/20,11/3/20,REP,Donald Trump,48.0
2,The Political Matrix/The Listener Group,11/2/20,11/2/20,DEM,Joe Biden,44.2
3,The Political Matrix/The Listener Group,11/2/20,11/2/20,REP,Donald Trump,48.0
4,PPP,11/1/20,11/2/20,DEM,Joe Biden,49.0


In [ ]:
poll_data.shape

(16715, 6)

In [ ]:
poll_data.isnull().sum()

,0
pollster,0
start_date,0
end_date,0
party,0
candidate_name,0
pct,0


In [ ]:
def rename_candidates(poll_data):
    # List of allowed candidate names
    allowed_candidates = ['Joe Biden', 'Donald Trump', 'Jo Jorgensen', 'Howie Hawkins',
                          'Kanye West', 'Don Blankenship', 'Roque De La Fuente',
                          'Jade Simmons', 'Brock Pierce', 'Gloria La Riva']

    # Get the list of unique candidate names before processing
    unique_candidates_before = poll_data['candidate_name'].unique()

    # Drop rows where the candidate_name is not in the allowed list
    poll_data = poll_data[poll_data['candidate_name'].isin(allowed_candidates)]

    # Replace candidate names: "Joe Biden" and "Donald Trump" remain as they are,
    # others are renamed to "Other Candidates"
    poll_data['candidate_name'] = poll_data['candidate_name'].apply(
        lambda x: 'Other Candidates' if x not in ['Joe Biden', 'Donald Trump'] else x
    )

    # Get the list of unique candidate names after processing
    unique_candidates_after = poll_data['candidate_name'].unique()

    # Get the unique values in the 'party' column
    unique_parties = poll_data['party'].unique()

    return poll_data, unique_candidates_before, unique_candidates_after, unique_parties

In [ ]:
# Call the function and unpack the returned values
poll_data, unique_candidates_before, unique_candidates_after, unique_parties = rename_candidates(poll_data)

In [ ]:
# Print unique candidate names before renaming
print("Unique candidate names before renaming:")
print(unique_candidates_before)

# Print unique candidate names after renaming
print("\nUnique candidate names after renaming:")
print(unique_candidates_after)

# Print unique values in the 'party' column
print("\nUnique values in the 'party' column:")
print(unique_parties)

Unique candidate names before renaming:
['Joe Biden' 'Donald Trump' 'Jo Jorgensen' 'Howie Hawkins' 'Kanye West'
 'Don Blankenship' 'Roque De La Fuente' 'Jade Simmons' 'Brock Pierce'
 'Mike Pence' 'Kamala Harris' 'Gloria La Riva' 'Alyson Kennedy'
 'Jacob George Hornberger' 'Andrew Cuomo' 'Hillary Rodham Clinton'
 'Barack Obama' 'Justin Amash' 'Bernard Sanders' 'Elizabeth Warren'
 'Michael Bloomberg' 'Pete Buttigieg' 'Amy Klobuchar' 'Tulsi Gabbard'
 'Tom Steyer' 'Andrew Yang' 'Michelle Obama' 'Cory A. Booker'
 'Julián Castro' "Beto O'Rourke" 'Nikki Haley' 'Steve Bullock'
 'John K. Delaney' 'Kirsten E. Gillibrand' 'Marianne Williamson'
 'Wayne Messam' 'Michael Bennet' 'Bill de Blasio' 'Oprah Winfrey'
 'Jay Robert Inslee' 'John Wright Hickenlooper' 'Mike Gravel'
 'Seth Moulton' 'Megan Rapinoe' 'Eric Swalwell' 'Tim Ryan'
 'Howard Schultz' 'Sherrod Brown' 'Nancy Pelosi' 'Charles E. Schumer'
 'Alexandria Ocasio-Cortez']

Unique candidate names after renaming:
['Joe Biden' 'Donald Trump' 'Othe

In [ ]:
poll_data.head()

,pollster,start_date,end_date,party,candidate_name,pct
0,Lake Research,10/31/20,11/3/20,DEM,Joe Biden,51.0
1,Lake Research,10/31/20,11/3/20,REP,Donald Trump,48.0
2,The Political Matrix/The Listener Group,11/2/20,11/2/20,DEM,Joe Biden,44.2
3,The Political Matrix/The Listener Group,11/2/20,11/2/20,REP,Donald Trump,48.0
4,PPP,11/1/20,11/2/20,DEM,Joe Biden,49.0


In [ ]:
poll_data.shape

(15005, 6)

In [ ]:
def convert_data_types(poll_data):
    # Convert 'start_date' and 'end_date' to datetime, inferring the format
    poll_data['start_date'] = pd.to_datetime(poll_data['start_date'], format='%m/%d/%y') # Changed format to match the date in the error message
    poll_data['end_date'] = pd.to_datetime(poll_data['end_date'], format='%m/%d/%y')  # Changed format to match the date in the error message
    poll_data['pct'] = pd.to_numeric(poll_data['pct'], errors='coerce')
    return poll_data

In [ ]:
poll_data = convert_data_types(poll_data)

In [ ]:
def add_additional_features(poll_data):
    # Add additional features
    poll_data['poll_duration'] = (poll_data['end_date'] - poll_data['start_date']).dt.days
    return poll_data

In [ ]:
poll_data = add_additional_features(poll_data)

In [ ]:
def cap_outliers_iqr(poll_data, column_name):

    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = poll_data[column_name].quantile(0.25)
    Q3 = poll_data[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = poll_data[(poll_data[column_name] < lower_bound) | (poll_data[column_name] > upper_bound)]
    num_outliers = len(outliers)

    # Cap the outliers at the lower and upper bounds
    poll_data[column_name] = np.where(poll_data[column_name] < lower_bound, lower_bound, poll_data[column_name])
    poll_data[column_name] = np.where(poll_data[column_name] > upper_bound, upper_bound, poll_data[column_name])

    # Output number of outliers capped
    print(f"Number of outliers capped in {column_name}: {num_outliers}")

    return poll_data, num_outliers

In [ ]:
poll_data, outliers_count = cap_outliers_iqr(poll_data, 'pct')

Number of outliers capped in pct: 1457


In [ ]:
poll_data.shape

(15005, 7)

In [ ]:
poll_data.head()

,pollster,start_date,end_date,party,candidate_name,pct,poll_duration
0,Lake Research,2020-10-31,2020-11-03,DEM,Joe Biden,51.0,3
1,Lake Research,2020-10-31,2020-11-03,REP,Donald Trump,48.0,3
2,The Political Matrix/The Listener Group,2020-11-02,2020-11-02,DEM,Joe Biden,44.2,0
3,The Political Matrix/The Listener Group,2020-11-02,2020-11-02,REP,Donald Trump,48.0,0
4,PPP,2020-11-01,2020-11-02,DEM,Joe Biden,49.0,1


In [ ]:
poll_data.isnull().sum()

,0
pollster,0
start_date,0
end_date,0
party,0
candidate_name,0
pct,0
poll_duration,0


In [ ]:
poll_data.to_csv('/content/drive/MyDrive/preprocessed_datasets/preprocessed_poll_data.csv', index = False)